In [ ]:
import os
os.environ['SUMO_HOME'] = '/usr/share/sumo'

In [ ]:
import time
from environment import create_env


env = create_env(tailgating=False)
total_reward = 0
state = env.reset()
for _ in range(100):
    actions = {
        ts_id: env.action_spaces(ts_id).sample()
        for ts_id in env.ts_ids
    }
    state, reward, done, _ = env.step(actions)
    total_reward += sum(reward.values())
    time.sleep(0.25)
    
print(f"Total reward: {total_reward}")

In [ ]:
env = create_env(tailgating=True)
total_reward = 0
state = env.reset()
for _ in range(100):
    actions = {
        ts_id: env.action_spaces(ts_id).sample()
        for ts_id in env.ts_ids
    }
    state, reward, done, _ = env.step(actions)
    total_reward += sum(reward.values())
    time.sleep(0.25)
    
print(f"Total reward: {total_reward}")

In [ ]:
from trainer import Trainer

trainer = Trainer(env='default')
trainer.train()

In [ ]:
agents = trainer.agents
env = create_env(tailgating=False)

total_reward = 0
state = env.reset()
for _ in range(100):
    actions = {
        ts_id: agents[ts_id].act(state[ts_id]) 
        for ts_id in state.keys()
    }
    state, reward, done, _ = env.step(actions)
    total_reward += sum(reward.values())
    time.sleep(0.25)
    
print(f"Total reward: {total_reward}")

In [ ]:
agents = trainer.agents
env = create_env(tailgating=True)

total_reward = 0
state = env.reset()
for _ in range(100):
    actions = {
        ts_id: agents[ts_id].act(state[ts_id]) 
        for ts_id in state.keys()
    }
    state, reward, done, _ = env.step(actions)
    total_reward += sum(reward.values())
    time.sleep(0.25)
    
print(f"Total reward: {total_reward}")